In [ ]:
import numpy as np

# Fixing the seed so results stay reproducible
np.random.seed(0)

def pm_sign(values):
    """
    Returns +1 for non-negative entries and -1 for negative ones.
    Works element-wise for arrays.
    """
    return np.where(values >= 0, 1, -1)


In [ ]:
# ----------------------------------------------------------------------
# Task 1: Hopfield network (10x10 memory grid)
# ----------------------------------------------------------------------

import numpy as np
np.random.seed(0)

def to_pm1(grid):
    """
    Convert a 10×10 grid of 0/1 values into a flat vector of {-1, +1}.
    """
    return 2 * grid.reshape(-1) - 1


def to_binary(vec):
    """
    Convert a {-1, +1} vector back to a 10×10 array of 0/1 values.
    """
    return ((vec + 1) // 2).astype(int).reshape(10, 10)


def hopfield_learn(pattern_list):
    """
    Build the Hopfield weight matrix using simple Hebbian accumulation.
    pattern_list: list of length-100 vectors containing {-1, +1}.
    """
    dim = pattern_list[0].size
    weights = np.zeros((dim, dim))

    for patt in pattern_list:
        weights += np.outer(patt, patt)

    np.fill_diagonal(weights, 0)     # remove self-connections
    weights /= dim                   # classic normalization
    return weights


def async_update(weights, state_vec, steps=2000):
    """
    Apply asynchronous recall: pick a random index each step and update it.
    """
    curr = state_vec.copy()
    length = curr.size

    for _ in range(steps):
        idx = np.random.randint(0, length)
        net = np.dot(weights[idx], curr)
        curr[idx] = 1 if net >= 0 else -1

    return curr


def run_demo():
    print("\n--- Hopfield 10×10 Memory Example ---")

    # Three simple shapes
    patt1 = np.zeros((10, 10), int)
    np.fill_diagonal(patt1, 1)

    patt2 = np.zeros((10, 10), int)
    patt2[0, :] = patt2[-1, :] = 1

    patt3 = np.zeros((10, 10), int)
    patt3[:, 0] = patt3[:, -1] = 1

    binary_list = [patt1, patt2, patt3]
    pm_list = [to_pm1(b) for b in binary_list]

    W = hopfield_learn(pm_list)

    # Distort the first pattern
    original = pm_list[0]
    noisy = original.copy()
    flip_ids = np.random.choice(len(noisy), 20, replace=False)
    noisy[flip_ids] *= -1

    restored = async_update(W, noisy, steps=3000)

    print("Original:")
    print(to_binary(original))
    print("Noisy:")
    print(to_binary(noisy))
    print("Recovered:")
    print(to_binary(restored))


run_demo()



--- Hopfield 10×10 Memory Example ---
Original:
[[1 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 1]]
Noisy:
[[1 0 1 0 0 0 0 1 1 0]
 [0 1 0 1 0 0 1 0 0 0]
 [0 0 0 0 1 0 1 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [0 0 0 1 1 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0]
 [0 0 0 1 0 1 0 1 1 0]
 [0 0 0 0 0 0 1 0 1 0]
 [0 0 1 1 0 1 0 0 0 1]]
Recovered:
[[1 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 1]]


In [ ]:
# ----------------------------------------------------------------------
# Task 2: Estimating memory capacity of a Hopfield network
# ----------------------------------------------------------------------

def estimate_capacity(num_units):
    """
    Rough rule-of-thumb: a Hopfield net can store about
    0.138 * N random, independent patterns reliably.
    """
    return 0.138 * num_units


def show_capacity_example():
    print("\n--- Hopfield Network Capacity Check ---")

    total_nodes = 100  # 10x10 grid
    approx_cap = estimate_capacity(total_nodes)

    print(f"Neuron count: {total_nodes}")
    print(f"Estimated capacity ≈ 0.138 × {total_nodes} = {approx_cap:.2f}")
    print(f"Rounded: {int(approx_cap)} storable patterns")

show_capacity_example()



--- Hopfield Network Capacity Check ---
Neuron count: 100
Estimated capacity ≈ 0.138 × 100 = 13.80
Rounded: 13 storable patterns


In [ ]:
# ----------------------------------------------------------------------
# Task 3: Testing how well the Hopfield net fixes corrupted patterns
# ----------------------------------------------------------------------

def random_bit_flips(vec_pm1, k):
    """
    Flip exactly k positions in a {-1, +1} vector.
    """
    modified = vec_pm1.copy()
    flip_positions = np.random.choice(modified.size, size=k, replace=False)
    modified[flip_positions] *= -1
    return modified


def recovery_success_rate(weight_matrix, saved_patterns, k, attempts=50):
    """
    Estimate how often the network successfully restores a pattern
    when exactly k bits are flipped.

    Each attempt:
      - choose one stored pattern at random
      - flip k entries
      - run asynchronous recall
      - check if final output matches original
    """
    successes = 0

    for _ in range(attempts):
        original = saved_patterns[np.random.randint(len(saved_patterns))]
        noisy = random_bit_flips(original, k)
        output = recall_async(weight_matrix, noisy, n_steps=2000)

        if np.array_equal(output, original):
            successes += 1

    return successes / attempts


def run_error_test():
    print("\n--- Error Correction Test (Hopfield Network) ---")

    # reuse simple 10×10 demo patterns
    pat1 = np.zeros((10, 10), dtype=int); np.fill_diagonal(pat1, 1)
    pat2 = np.zeros((10, 10), dtype=int); pat2[0, :], pat2[-1, :] = 1, 1
    pat3 = np.zeros((10, 10), dtype=int); pat3[:, 0], pat3[:, -1] = 1, 1

    binary_patterns = [pat1, pat2, pat3]
    vec_patterns = [bin_to_pm1(p) for p in binary_patterns]

    W = train_hopfield(vec_patterns)

    # test network for increasing corruption levels
    for k in [0, 5, 10, 15, 20, 25, 30]:
        score = recovery_success_rate(W, vec_patterns, k, attempts=20)
        print(f"Flips = {k} → success ≈ {score*100:.1f}%")

    print("Higher corruption generally reduces recovery accuracy.")

run_error_test()



--- Error Correction Test (Hopfield Network) ---
Flips = 0 → success ≈ 100.0%
Flips = 5 → success ≈ 70.0%
Flips = 10 → success ≈ 65.0%
Flips = 15 → success ≈ 80.0%
Flips = 20 → success ≈ 65.0%
Flips = 25 → success ≈ 25.0%
Flips = 30 → success ≈ 35.0%
Higher corruption generally reduces recovery accuracy.


In [ ]:
# ----------------------------------------------------------------------
# Task 4: Eight-rook placement via Hopfield-inspired energy minimization
# ----------------------------------------------------------------------

def rook_cost(grid):
    """
    Compute energy of an 8×8 rook placement.
    Each row and each column should contain exactly one rook.
    """
    weight_rows = 1.0
    weight_cols = 1.0

    r_totals = grid.sum(axis=1)
    c_totals = grid.sum(axis=0)

    penalty_rows = weight_rows * np.sum((r_totals - 1) ** 2)
    penalty_cols = weight_cols * np.sum((c_totals - 1) ** 2)

    return penalty_rows + penalty_cols


def search_eight_rooks(steps=10000):
    """
    Greedy search: flip board cells only if the energy does not increase.
    """
    # random 8×8 start (0 = empty, 1 = rook)
    layout = (np.random.rand(8, 8) > 0.5).astype(int)
    cur_energy = rook_cost(layout)

    for _ in range(steps):
        changed = False

        # try flipping each cell once
        for r in range(8):
            for c in range(8):
                layout[r, c] ^= 1
                new_energy = rook_cost(layout)

                if new_energy <= cur_energy:
                    cur_energy = new_energy
                    changed = True
                else:
                    layout[r, c] ^= 1  # undo flip

        if not changed:  # stuck at local optimum
            break

    return layout, cur_energy


def run_rook_demo():
    print("\n--- Eight-Rook Hopfield-style Solver ---")

    config, final_E = search_eight_rooks(steps=20000)

    print("Board (1 = rook):")
    print(config)
    print("Row counts :", config.sum(axis=1))
    print("Column counts :", config.sum(axis=0))
    print("Energy:", final_E)

    if np.all(config.sum(axis=1) == 1) and np.all(config.sum(axis=0) == 1):
        print("✔ Valid non-attacking placement achieved.")
    else:
        print("✘ Ended in local minimum — try running again.")

run_rook_demo()



--- Eight-Rook Hopfield-style Solver ---
Board (1 = rook):
[[0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 1 0]
 [1 0 0 0 0 0 0 1]
 [0 1 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0]
 [0 0 0 1 0 0 0 0]]
Row counts : [1 1 1 1 2 1 1 1]
Column counts : [1 1 1 2 1 1 1 1]
Energy: 2.0
✘ Ended in local minimum — try running again.


In [ ]:
# ----------------------------------------------------------------------
# Task 5: Solving a 10-city TSP using a Hopfield–Tank style network
# ----------------------------------------------------------------------

def run_tsp_network(dmat, 
                    alpha=500.0, beta=500.0, gamma=200.0,
                    lr=0.01, iterations=5000):
    """
    Continuous Hopfield–Tank dynamics for TSP.
    
    dmat  : symmetric distance matrix (nxn).
    alpha : penalty for repeating a city.
    beta  : penalty for filling a time slot more than once.
    gamma : contribution of travel distance.
    """
    n = dmat.shape[0]

    # internal potentials
    U = np.random.randn(n, n)
    # neuron activations (0..1)
    V = 0.5 * (1 + np.tanh(U))

    for _ in range(iterations):
        grad = np.zeros((n, n))

        # Each city should appear once
        for city in range(n):
            s = V[city].sum()
            grad[city] += 2 * alpha * (s - 1)

        # Each slot should contain exactly one city
        for pos in range(n):
            s = V[:, pos].sum()
            grad[:, pos] += 2 * beta * (s - 1)

        # Travel distance contribution
        for pos in range(n):
            nxt = (pos + 1) % n
            for city in range(n):
                grad[city, pos] += gamma * np.sum(dmat[city] * V[:, nxt])

        # update potentials and outputs
        U -= lr * grad
        V = 0.5 * (1 + np.tanh(U))

    return V


def extract_route(Vmap):
    """
    Convert soft V[x,i] to a final tour
    by choosing the city with max activation at each position.
    """
    return Vmap.argmax(axis=0)


def route_length(route, dist):
    """
    Compute total length of the cyclic route.
    """
    total = 0.0
    n = len(route)
    for i in range(n):
        a = route[i]
        b = route[(i + 1) % n]
        total += dist[a, b]
    return total


def tsp_demo():
    print("\n--- Hopfield–Tank TSP Demo (10 Cities) ---")

    n = 10
    neurons = n * n
    possible_edges = neurons * (neurons - 1) // 2

    print(f"Total neurons: {neurons}")
    print(f"Unique symmetric weights: {possible_edges}")

    # random symmetric distances
    D = np.random.rand(n, n)
    D = (D + D.T) / 2
    np.fill_diagonal(D, 0)

    print("\nDistance matrix:")
    print(np.round(D, 2))

    V_end = run_tsp_network(D, iterations=4000)
    tour = extract_route(V_end)
    total_len = route_length(tour, D)

    print("\nChosen route (0–9):")
    print(tour)
    print(f"Total distance: {total_len:.3f}")


tsp_demo()



--- Hopfield–Tank TSP Demo (10 Cities) ---
Total neurons: 100
Unique symmetric weights: 4950

Distance matrix:
[[0.   0.36 0.36 0.27 0.56 0.47 0.51 0.58 0.61 0.55]
 [0.36 0.   0.61 0.73 0.28 0.27 0.4  0.63 0.81 0.44]
 [0.36 0.61 0.   0.35 0.27 0.49 0.49 0.53 0.32 0.59]
 [0.27 0.73 0.35 0.   0.31 0.59 0.06 0.22 0.49 0.94]
 [0.56 0.28 0.27 0.31 0.   0.45 0.4  0.51 0.31 0.48]
 [0.47 0.27 0.49 0.59 0.45 0.   0.48 0.95 0.16 0.31]
 [0.51 0.4  0.49 0.06 0.4  0.48 0.   0.46 0.19 0.29]
 [0.58 0.63 0.53 0.22 0.51 0.95 0.46 0.   0.31 0.38]
 [0.61 0.81 0.32 0.49 0.31 0.16 0.19 0.31 0.   0.45]
 [0.55 0.44 0.59 0.94 0.48 0.31 0.29 0.38 0.45 0.  ]]

Chosen route (0–9):
[7 3 6 8 5 4 4 2 2 9]
Total distance: 2.315
